# ETL - Mounting Data from Azure datalake

## Mount Storage

In [0]:
dbutils.fs.mount(
  source = "wasbs://vscontainer@azdedlvsp.blob.core.windows.net",
  mount_point = "/mnt/sample/sale",
  extra_configs = {"fs.azure.account.key.azdedlvsp.blob.core.windows.net":"6+LqE3+iijfVReWMdHWrjIbjHqsUK3VlNjGcIMTGTrInY2udPS1zjPOkKV99/bd2URGKuV7LZALJ+ASthg70vA=="})


True

## Extract

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("ETL pipeline").getOrCreate()

# Define the file path
file_path = "/mnt/sample/sale/sales.csv"

# Read the CSV file
df = spark.read.csv(file_path, header=True, inferSchema=True)


In [0]:
display(df)

Date,Day,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,State,Product_Category,Sub_Category,Product,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue
2013-11-26,26,November,2013,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
2015-11-26,26,November,2015,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
2014-03-23,23,March,2014,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,23,45,120,1366,1035,2401
2016-03-23,23,March,2016,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,20,45,120,1188,900,2088
2014-05-15,15,May,2014,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,238,180,418
2016-05-15,15,May,2016,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,5,45,120,297,225,522
2014-05-22,22,May,2014,47,Adults (35-64),F,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,199,180,379
2016-05-22,22,May,2016,47,Adults (35-64),F,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,2,45,120,100,90,190
2014-02-22,22,February,2014,35,Adults (35-64),M,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,22,45,120,1096,990,2086
2016-02-22,22,February,2016,35,Adults (35-64),M,Australia,Victoria,Accessories,Bike Racks,Hitch Rack - 4-Bike,21,45,120,1046,945,1991


## Transform

In [0]:
# Drop the specified columns
columns_to_drop = ["Date", "Day", "Customer_Age"]
transformed_df = df.drop(*columns_to_drop)

# Drop duplicate records from the DataFrame
deduplicated_df = transformed_df.dropDuplicates()
male_df = deduplicated_df.filter(deduplicated_df.Customer_Gender == "M")

In [0]:
df.count()

189

In [0]:
male_df.count()

87

## Load

In [0]:
# Specify the target path in Azure Data Lake
output_path = "/mnt/sample/sale/sale_M.csv"

# Write the DataFrame as a CSV file
male_df.write.mode("overwrite").option("header", "true").csv(output_path)